# PERCEPTRÓN PARA CLASIFICACIÓN - (UNA SOLA NEURONA) - CON FUNCIÓN DE ACTIVACIÓN SIGMOIDAL

## Alexander Alzate - Mateo Arroyave
### Obteniendo y normalizando datos

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
datos_xls = pd.ExcelFile('DatosPunto2.xlsx') #Obteniendo el archivo
sheet_names = datos_xls.sheet_names #Obteniendo un arreglo con los nombres de las hojas del libro de excel
hoja = datos_xls.parse(sheet_names[0]) #Nuestra hoja de trabajo es la primera pestaña
#print(hoja)
columnas = [] #Una lista vacia de columnas 
columnas_norm = [] #Una lista vacia de las columnas normalizadas
for i in range(1,17):      
    columnas.append(hoja.iloc[:,i]) #Se agrega a la lista el elemento = a un arreglo o columna 
    columnas_norm.append((columnas[i-1]-np.min(columnas[i-1],axis=0))/(np.max(columnas[i-1],axis=0)-np.min(columnas[i-1],axis=0)))
#print(columnas_norm)
matriz_norm = np.array(columnas_norm)#[16,20000]
matriz_norm = np.array(matriz_norm.T)#[20000,16]
#print(np.shape(matriz_norm))#[20000,16]
#print("la matriz normalizada es:\n ",matriz_norm)

### Obtenemos la clasificación de los datos (Valores: Clase1: -1, Clase2: 1 ; [-1,1])

In [4]:
#Etiquetados: 1 si la letra esta dentro de mi apellido, -1 caso contrario
vect_letras=['A','L','Z','T','E','R','O','V','Y'] #Vector para definir cuales son los de la clase 1
lista_etiqueta=[]#Se crea una lista etiquetando en -1 y 1
matriz_datos=np.array(hoja)
#print(np.shape(matriz_datos))
for i in range (len(matriz_datos)): #For que va desde i hasta el tamaño de la matriz de datos
    for j in range (len(vect_letras)): #Recorriendo el vector de letras
        if(matriz_datos[i,0]==vect_letras[j]): #Comparando si la fila de los datos en la columna 0 coincide con una letra del vector donde estan las letras de apellidos
            flag_clase1=True #Pertenece a la clase 1
            break #Dejar de recorrer el vector letras del apellido, si encuentra la letra
        else:
            flag_clase1=False #Sino es una etiqeta cualquiera
    if(flag_clase1): #Si se levanto la bandera que es entonces las etiquetas se colocan en 1
        lista_etiqueta.append(1)
    else:
        lista_etiqueta.append(-1) #Sino en -1
print(matriz_datos[0:18,0])#primera fila
print(lista_etiqueta[0:18])

['T' 'I' 'D' 'N' 'G' 'S' 'B' 'A' 'J' 'M' 'X' 'O' 'G' 'M' 'R' 'F' 'O' 'C']
[1, -1, -1, -1, -1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1]


### Dividir la matriz de datos de entrada para entrenamiento, validación y prueba

In [10]:
Numero_Filas_Datos = len(matriz_norm[:,0]) #Obteniendo la cantidad de elementos en una lista, es decir la cantidad de filas
print("Numero de filas: ",Numero_Filas_Datos)
#Se realiza la división para hacer bootstraping
N_datos_entrenamiento = int(Numero_Filas_Datos*0.7) #70% para entrenamiento 
N_datos_validacion = int(Numero_Filas_Datos*0.15) #15% para validación
N_datos_prueba = Numero_Filas_Datos - N_datos_validacion - N_datos_entrenamiento #15% para prueba
#print("N_datos_entrenamiento: ",N_datos_entrenamiento)
#print("N_datos_validacion: ",N_datos_validacion)
#print("N_datos_prueba: ",N_datos_prueba)

#Ya tenemos una matriz normalizada anteriormente llamada matriz_norm[20000,16]
vector_etiquetas=np.array(lista_etiqueta) #Pasarlo a vector
print ("vector etiquetas: ",np.shape(vector_etiquetas)) #[20000,1]
matriz_entrenamiento = matriz_norm[0:N_datos_entrenamiento,:] #Matriz[14000,16]
print("matriz_entrenamiento: ",matriz_entrenamiento.shape)
vector_etiquetas_entrenamiento = vector_etiquetas[0:N_datos_entrenamiento] #Matriz[14000,1]
print("vector_etiquetas_entrenamiento: ",vector_etiquetas_entrenamiento.shape)
matriz_validacion = matriz_norm[N_datos_entrenamiento:(N_datos_entrenamiento+N_datos_validacion),:] #Matriz[3000,16]
print("matriz_validacion: ",matriz_validacion.shape)
vector_etiquetas_validacion = vector_etiquetas[N_datos_entrenamiento:(N_datos_entrenamiento+N_datos_validacion)] #Matriz[3000,1]
print("vector_etiquetas_validacion: ",vector_etiquetas_validacion.shape)
matriz_prueba = matriz_norm[N_datos_validacion:(N_datos_validacion+N_datos_prueba),:] #Matriz[3000,16]
print("matriz_prueba: ",matriz_prueba.shape)
vector_etiquetas_prueba = vector_etiquetas[N_datos_validacion:(N_datos_validacion+N_datos_prueba)] #Matriz[3000,1]
print("vector_etiquetas_prueba: ",vector_etiquetas_prueba.shape)

Numero de filas:  20000
vector etiquetas:  (20000,)
matriz_entrenamiento:  (14000, 16)
vector_etiquetas_entrenamiento:  (14000,)
matriz_validacion:  (3000, 16)
vector_etiquetas_validacion:  (3000,)
matriz_prueba:  (3000, 16)
vector_etiquetas_prueba:  (3000,)


### Definiendo función de activación para implementar el algoritmo perceptrón

In [11]:
#Definición de la función sigmoidal
def sigmoid(x):
    return 1/(1+np.exp(-x))

### Implementando algoritmo Perceptron para entrenamiento y validación

In [12]:
def Perceptron_Train(matriz_train, vector_etiquetas_entrenamiento, max_iteraciones = 3000, eficiencia_min = 0.80):
    contador_iteraciones = 0 #cuenta el numero de iteraciones para obtner luego una rata de error 
    contador_errores_train = 0 #cuenta los datos que fueron mal clasificados durante la iteración con la matriz de entrenamiento
    result_valid = False #Resultado de la validación, false si no valida y true si valida
    flag_culm = True #indica el estado dela iteración del algoritmo
    vector_pesos = np.random.rand(matriz_train.shape[1] + 1,) #se crea el vector de pesos inicial aleatorio
    while(flag_culm):
        contador_iteraciones = contador_iteraciones + 1;
        dato_escogido = np.random.randint(0,N_datos_entrenamiento) #Escogiendo un numero para indicar que fila de datos escoger
        g  = np.dot(vector_pesos.T,np.insert(matriz_train[dato_escogido,:],0,1)) # y = w.xT donde Y = matriz de salida, wT = Matriz de pesos
        yp = sigmoid(g)-0.5#Pasando por la funcion de activación que se le baja su offset de 0.5 
        result_comparacion = yp*vector_etiquetas_entrenamiento[dato_escogido]
        #if( (yp >= 0 & vector_etiquetas_entrenamiento[dato_escogido] < 0) | (yp <= 0 & vector_etiquetas_entrenamiento[dato_escogido]> 0)):
        if result_comparacion < 0:
            contador_errores_train = contador_errores_train +1
            vector_pesos = vector_pesos + np.dot(np.insert(matriz_train[dato_escogido,:],0,1),vector_etiquetas_entrenamiento[dato_escogido]) #actualización del vector de pesos 
        result_valid = func_valid(vector_pesos,matriz_validacion, vector_etiquetas_validacion, eficiencia_min)
        if(contador_iteraciones == max_iteraciones):
            flag_culm = False #Dejar de iterar
            print("culminacion por iteraciones")
        if(result_valid):
            print("culminacion por validacion")
            flag_culm = False #Dejar de iterar
    print("el numero de iteraciones fue: ",contador_iteraciones)
    #print("contador de errores train: ",contador_errores_train)
    print("El vector de pesos optimo W es:\n")
    for i in range(0,len(vector_pesos)):
        print("W[",i,"]=",vector_pesos[i])
    return vector_pesos

def func_valid(vector_pesos,matriz_validacion, vector_etiquetas_validacion, eficiencia_min):
    #nuestro criterio de parada será una eficiencia respecto al numero de datos analizados
    contador_errores_valid = 0
    matriz_validacion_bias = np.insert(matriz_validacion,0,1,axis=1)
    yp_validacion = np.dot(matriz_validacion_bias,vector_pesos.T)
    for i in range(0,len(yp_validacion)):
        g = sigmoid(yp_validacion[i])-0.5
        if g*vector_etiquetas_validacion[i] < 0: #clasificacion erronea
            contador_errores_valid = contador_errores_valid + 1 #para llevar el conteo de los datos clasificados erroneamente
            
    print("contador de errores validacion: ",contador_errores_valid)
    eficiencia = (len(matriz_validacion)-contador_errores_valid)/len(matriz_validacion)
    print("\nla eficiencia total en validacion fue: ",eficiencia)
    
    if eficiencia >= eficiencia_min:
        flag_valid = True
    else:
        flag_valid = False
    return flag_valid    
        

In [8]:
vector_pesos_final = Perceptron_Train(matriz_entrenamiento, vector_etiquetas_entrenamiento, max_iteraciones = 3000, eficiencia_min = 0.71)
#print("el vector de pesos final es: ",vector_pesos_final)

contador de errores validacion:  1935

la eficiencia total en validacion fue:  0.355
contador de errores validacion:  1935

la eficiencia total en validacion fue:  0.355
contador de errores validacion:  1935

la eficiencia total en validacion fue:  0.355
contador de errores validacion:  1935

la eficiencia total en validacion fue:  0.355
contador de errores validacion:  1067

la eficiencia total en validacion fue:  0.6443333333333333
contador de errores validacion:  1067

la eficiencia total en validacion fue:  0.6443333333333333
contador de errores validacion:  1935

la eficiencia total en validacion fue:  0.355
contador de errores validacion:  1935

la eficiencia total en validacion fue:  0.355
contador de errores validacion:  1065

la eficiencia total en validacion fue:  0.645
contador de errores validacion:  1065

la eficiencia total en validacion fue:  0.645
contador de errores validacion:  1935

la eficiencia total en validacion fue:  0.355
contador de errores validacion:  1453



contador de errores validacion:  1065

la eficiencia total en validacion fue:  0.645
contador de errores validacion:  1065

la eficiencia total en validacion fue:  0.645
contador de errores validacion:  998

la eficiencia total en validacion fue:  0.6673333333333333
contador de errores validacion:  1065

la eficiencia total en validacion fue:  0.645
contador de errores validacion:  1065

la eficiencia total en validacion fue:  0.645
contador de errores validacion:  985

la eficiencia total en validacion fue:  0.6716666666666666
contador de errores validacion:  985

la eficiencia total en validacion fue:  0.6716666666666666
contador de errores validacion:  985

la eficiencia total en validacion fue:  0.6716666666666666
contador de errores validacion:  985

la eficiencia total en validacion fue:  0.6716666666666666
contador de errores validacion:  1917

la eficiencia total en validacion fue:  0.361
contador de errores validacion:  1034

la eficiencia total en validacion fue:  0.655333333

contador de errores validacion:  1056

la eficiencia total en validacion fue:  0.648
contador de errores validacion:  1056

la eficiencia total en validacion fue:  0.648
contador de errores validacion:  1056

la eficiencia total en validacion fue:  0.648
contador de errores validacion:  1761

la eficiencia total en validacion fue:  0.413
contador de errores validacion:  999

la eficiencia total en validacion fue:  0.667
contador de errores validacion:  999

la eficiencia total en validacion fue:  0.667
contador de errores validacion:  999

la eficiencia total en validacion fue:  0.667
contador de errores validacion:  999

la eficiencia total en validacion fue:  0.667
contador de errores validacion:  999

la eficiencia total en validacion fue:  0.667
contador de errores validacion:  999

la eficiencia total en validacion fue:  0.667
contador de errores validacion:  1718

la eficiencia total en validacion fue:  0.42733333333333334
contador de errores validacion:  1718

la eficiencia tota

contador de errores validacion:  1722

la eficiencia total en validacion fue:  0.426
contador de errores validacion:  1722

la eficiencia total en validacion fue:  0.426
contador de errores validacion:  962

la eficiencia total en validacion fue:  0.6793333333333333
contador de errores validacion:  962

la eficiencia total en validacion fue:  0.6793333333333333
contador de errores validacion:  962

la eficiencia total en validacion fue:  0.6793333333333333
contador de errores validacion:  962

la eficiencia total en validacion fue:  0.6793333333333333
contador de errores validacion:  962

la eficiencia total en validacion fue:  0.6793333333333333
contador de errores validacion:  1636

la eficiencia total en validacion fue:  0.45466666666666666
contador de errores validacion:  1934

la eficiencia total en validacion fue:  0.35533333333333333
contador de errores validacion:  1595

la eficiencia total en validacion fue:  0.4683333333333333
contador de errores validacion:  1595

la eficien

contador de errores validacion:  1929

la eficiencia total en validacion fue:  0.357
contador de errores validacion:  1929

la eficiencia total en validacion fue:  0.357
contador de errores validacion:  1085

la eficiencia total en validacion fue:  0.6383333333333333
contador de errores validacion:  1085

la eficiencia total en validacion fue:  0.6383333333333333
contador de errores validacion:  1085

la eficiencia total en validacion fue:  0.6383333333333333
contador de errores validacion:  1085

la eficiencia total en validacion fue:  0.6383333333333333
contador de errores validacion:  1055

la eficiencia total en validacion fue:  0.6483333333333333
contador de errores validacion:  1055

la eficiencia total en validacion fue:  0.6483333333333333
contador de errores validacion:  1055

la eficiencia total en validacion fue:  0.6483333333333333
contador de errores validacion:  1182

la eficiencia total en validacion fue:  0.606
contador de errores validacion:  1182

la eficiencia total 

contador de errores validacion:  939

la eficiencia total en validacion fue:  0.687
contador de errores validacion:  1740

la eficiencia total en validacion fue:  0.42
contador de errores validacion:  944

la eficiencia total en validacion fue:  0.6853333333333333
contador de errores validacion:  1724

la eficiencia total en validacion fue:  0.42533333333333334
contador de errores validacion:  1724

la eficiencia total en validacion fue:  0.42533333333333334
contador de errores validacion:  901

la eficiencia total en validacion fue:  0.6996666666666667
contador de errores validacion:  901

la eficiencia total en validacion fue:  0.6996666666666667
contador de errores validacion:  901

la eficiencia total en validacion fue:  0.6996666666666667
contador de errores validacion:  901

la eficiencia total en validacion fue:  0.6996666666666667
contador de errores validacion:  1570

la eficiencia total en validacion fue:  0.4766666666666667
contador de errores validacion:  1570

la eficienci

### Visualización de elemento final 

In [9]:
matriz_prueba_bias = np.insert(matriz_prueba,0,1,axis=1) # se agrega el bias a la matriz de validación
prediccion_prueba = np.dot(matriz_prueba_bias,vector_pesos_final) #se hace una predicción del elemento a clasificar
cont_final=0
for i in range(len(prediccion_prueba)): # se recorren todos los datos de la matriz de prueba
    if ((sigmoid((prediccion_prueba[i]*vector_etiquetas_prueba[i])))-0.5) < 0: # se evalúa el estado de la clasificación
        cont_final = cont_final+1 #se cuenta el numero de clasificaciones incorrectas   
print("la eficiencia final es: ",((len(matriz_prueba_bias)-cont_final)/len(matriz_prueba_bias))) # se obtiene la eficiencia total

la eficiencia final es:  0.7113333333333334
